In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    roc_auc_score, f1_score, precision_score, recall_score,
    classification_report, make_scorer
)
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

# Paths
DATA_PATH = '../data/train_numeric.csv'
MODEL_PATH = '../models/'
SAMPLE_SIZE = 100_000

print("📦 Kütüphaneler yüklendi")

## 1. Veri Yükleme ve Hazırlık

In [ ]:
# Veri yükleme
print("📊 Veri yükleniyor...")
df = pd.read_csv(DATA_PATH, nrows=SAMPLE_SIZE)
print(f"Veri boyutu: {df.shape}")

# Target
y = df['Response']
X = df.drop(['Response', 'Id'], axis=1)

# Eksik veri temizleme (>90% missing olan sütunları kaldır)
missing_ratio = X.isnull().sum() / len(X)
cols_to_drop = missing_ratio[missing_ratio > 0.90].index.tolist()
X_cleaned = X.drop(columns=cols_to_drop)

# Kalan eksik verileri doldur
X_cleaned = X_cleaned.fillna(-999)

print(f"\n✅ Temizlenmiş veri: {X_cleaned.shape}")
print(f"🎯 Hedef dağılımı: {y.value_counts().to_dict()}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_cleaned, y, test_size=0.2, random_state=42, stratify=y
)
print(f"\nTrain: {X_train.shape}, Test: {X_test.shape}")

## 2. Hiperparametre Arama Uzayı

XGBoost için önemli parametreler:
- `max_depth`: Ağaç derinliği (overfitting kontrolü)
- `learning_rate`: Öğrenme hızı
- `n_estimators`: Ağaç sayısı
- `min_child_weight`: Minimum yaprak ağırlığı
- `subsample`: Satır örnekleme oranı
- `colsample_bytree`: Sütun örnekleme oranı
- `scale_pos_weight`: Dengesiz sınıf ağırlığı

In [ ]:
# Sınıf dengesizliği ağırlığı
scale_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"⚖️ Scale pos weight: {scale_weight:.2f}")

# Parametre arama uzayı (hızlı arama için daraltılmış)
param_grid = {
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Toplam kombinasyon sayısı
total_combinations = 1
for v in param_grid.values():
    total_combinations *= len(v)
print(f"\n🔍 Toplam parametre kombinasyonu: {total_combinations}")
print(f"⏱️ Tahmini süre: ~{total_combinations * 2} dakika (3-fold CV)")

In [ ]:
# Zaman kısıtı nedeniyle daha küçük bir grid kullanalım
param_grid_small = {
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1],
    'n_estimators': [100, 200],
    'min_child_weight': [1, 3]
}

total_small = 1
for v in param_grid_small.values():
    total_small *= len(v)
print(f"🎯 Optimize edilmiş grid: {total_small} kombinasyon")

## 3. GridSearchCV ile Optimizasyon

In [ ]:
# Base model
base_model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    scale_pos_weight=scale_weight,
    random_state=42,
    n_jobs=-1,
    use_label_encoder=False
)

# Stratified K-Fold (dengesiz veri için şart)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# F1 scorer (ana metriğimiz)
f1_scorer = make_scorer(f1_score)

print("🔄 GridSearchCV başlatılıyor...")
print("⏳ Bu işlem birkaç dakika sürebilir...\n")

grid_search = GridSearchCV(
    estimator=base_model,
    param_grid=param_grid_small,
    scoring='roc_auc',  # AUC daha stabil
    cv=cv,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

In [ ]:
# En iyi parametreler
print("\n" + "="*50)
print("🏆 EN İYİ PARAMETRELER")
print("="*50)
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\n📊 En iyi CV skoru (AUC): {grid_search.best_score_:.4f}")

## 4. En İyi Model ile Test

In [ ]:
# En iyi model
best_model = grid_search.best_estimator_

# Test seti tahminleri
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Metrikler
print("\n" + "="*50)
print("📈 TEST SETİ SONUÇLARI")
print("="*50)
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")

print("\n" + "="*50)
print("Classification Report:")
print("="*50)
print(classification_report(y_test, y_pred, target_names=['Sağlam', 'Hatalı']))

## 5. Parametre Sonuçları Tablosu

In [ ]:
# Tüm sonuçları DataFrame olarak göster
results_df = pd.DataFrame(grid_search.cv_results_)

# En iyi 10 sonuç
top_results = results_df.nsmallest(10, 'rank_test_score')[[
    'param_max_depth', 'param_learning_rate', 'param_n_estimators',
    'param_min_child_weight', 'mean_test_score', 'std_test_score', 'rank_test_score'
]]

print("\n🏅 En İyi 10 Parametre Kombinasyonu:")
print(top_results.to_string(index=False))

## 6. Optimizasyon Sonucu Kaydet

In [ ]:
# Optimum parametreleri kaydet
optimization_results = {
    'best_params': grid_search.best_params_,
    'best_cv_score': grid_search.best_score_,
    'test_auc': roc_auc_score(y_test, y_pred_proba),
    'test_f1': f1_score(y_test, y_pred),
    'test_precision': precision_score(y_test, y_pred),
    'test_recall': recall_score(y_test, y_pred)
}

with open(f'{MODEL_PATH}optimization_results.pkl', 'wb') as f:
    pickle.dump(optimization_results, f)

print("\n✅ Optimizasyon sonuçları kaydedildi: models/optimization_results.pkl")

## 7. Özet

### Yapılanlar:
1. ✅ GridSearchCV ile hiperparametre araması
2. ✅ Stratified K-Fold cross-validation
3. ✅ En iyi parametrelerin belirlenmesi
4. ✅ Test seti değerlendirmesi

### Sonraki Adım:
- `05_pipeline_final.ipynb` - Final pipeline oluşturma ve SHAP analizi